In [1]:
#import packages
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from pandas import Series,DataFrame
from matplotlib import pyplot as plt
import math

In [2]:
#read data
##airbnb data, from inside airbnb
airbnb = pd.read_csv("data/listings.csv")
##land use data
landuse_url = 'https://raw.githubusercontent.com/luzhao2000/fsdsA2_data/main/Live_Tables_-_Land_Use_Stock_2022_-_LSOA.csv'
landuse = pd.read_csv(landuse_url,skiprows=5)

##dwelling type data, from Census2021 https://www.ons.gov.uk/datasets/TS044/editions/2021/versions/1/filter-outputs/5956ae85-5004-4ee0-9498-87ed7e7f886a#get-data
dwelling_url = 'https://raw.githubusercontent.com/luzhao2000/fsdsA2_data/main/dwelling%20data.csv'
dwelling = pd.read_csv(dwelling_url)

##ownership data, from 
ownership_url = 'https://raw.githubusercontent.com/luzhao2000/fsdsA2_data/main/census2021.csv'
ownership = pd.read_csv(ownership_url)

##London shp, from London Datastore, https://data.london.gov.uk/dataset/statistical-gis-boundary-files-london
shp_url = 'https://github.com/luzhao2000/fsdsA2_data/blob/main/LSOA2011LondongenMHW.shp.zip?raw=true'
London = gpd.GeoDataFrame.from_file(shp_url)

/tmp/ipykernel_116/576169641.py:6: DtypeWarning: Columns (18,19,20,23,25,28,30,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  landuse = pd.read_csv(landuse_url,skiprows=5)


In [3]:
#data cleaning
##airbnb data, calculate counts and densities in each area of LSOA level
##na check
# for i in list(airbnb):
#     num=airbnb[airbnb[i].isnull()].shape[0]
#     print(f"{i} has {num} NaN.") #check whether there is na value in latitude and longitude fields, no.
#根据经纬度坐标，将属性数据转化为点，用于后面spatial join
airbnb_points = gpd.GeoDataFrame(airbnb,   geometry=gpd.points_from_xy(airbnb.longitude, airbnb.latitude))
airbnb_points.set_crs(epsg=4326)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,geometry
0,13913,https://www.airbnb.com/rooms/13913,20220910194334,2022-09-11,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.75,4.82,NaN,f,2,1,1,0,0.20,POINT (-0.11270 51.56861)
1,15400,https://www.airbnb.com/rooms/15400,20220910194334,2022-09-11,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.93,4.73,NaN,f,1,1,0,0,0.57,POINT (-0.16813 51.48780)
2,284532,https://www.airbnb.com/rooms/284532,20220910194334,2022-09-11,city scrape,COSY STUDIO-FLAT WITH A GREAT VIEW,My very RECENTLY RENOVATED flat is located in ...,Shoreditch is simply the coolest area in London!,https://a0.muscache.com/pictures/2932872/00037...,1481412,...,4.72,4.31,NaN,f,1,1,0,0,0.34,POINT (-0.07426 51.52851)
3,106332,https://www.airbnb.com/rooms/106332,20220910194334,2022-09-11,city scrape,"Lovely large room, Bethnal Green",Lovely characterful room in a house belonging ...,NaN,https://a0.muscache.com/pictures/82437180/20b7...,264345,...,4.92,4.83,NaN,f,1,0,1,0,0.47,POINT (-0.05396 51.52874)
4,17402,https://www.airbnb.com/rooms/17402,20220910194334,2022-09-11,city scrape,Superb 3-Bed/2 Bath & Wifi: Trendy W1,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",https://a0.muscache.com/pictures/39d5309d-fba7...,67564,...,4.88,4.59,NaN,f,5,5,0,0,0.36,POINT (-0.14094 51.52195)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69346,9671724,https://www.airbnb.com/rooms/9671724,20220910194334,2022-09-11,previous scrape,"Cozy room in Greenwich, river view","Single room available in a new three bedroom, ...",The following attractions can be found in Gree...,https://a0.muscache.com/pictures/29f1fa4e-299c...,49977905,...,4.86,4.93,NaN,f,3,1,2,0,0.84,POINT (-0.01346 51.48300)
69347,20277958,https://www.airbnb.com/rooms/20277958,20220910194334,2022-09-12,previous scrape,River view Tate Modern,"Best location in London, overlooking Thames Ri...","Best location in London, overlooking Thames Ri...",https://a0.muscache.com/pictures/pro_photo_too...,1132775,...,5.00,4.80,NaN,f,1,1,0,0,0.99,POINT (-0.10192 51.50914)
69348,20512703,https://www.airbnb.com/rooms/20512703,20220910194334,2022-09-12,previous scrape,"Luxury 4 bed house, 100m to river and palace",Forget your worries in this spacious and seren...,"Quiet, community suburb of London. River comes...",https://a0.muscache.com/pictures/4e5fed61-7fc4...,1706966,...,5.00,5.00,NaN,f,1,1,0,0,1.49,POINT (-0.35325 51.40571)
69349,49698549,https://www.airbnb.com/rooms/49698549,20220910194334,2022-09-12,previous scrape,Waterfront Hampton Stay,This luxury waterfront home is located on a ti...,Hampton is located on the outskirts of London ...,https://a0.muscache.com/pictures/44b6e0a6-ffae...,34213667,...,5.00,4.78,NaN,t,1,1,0,0,0.61,POINT (-0.35975 51.41029)


In [9]:
##london shp
london_proj = London.to_crs(epsg=4326) #transform CRS
london_proj_1 = london_proj.loc[:,['LSOA11CD','LSOA11NM','geometry']]
london_proj_2 = london_proj_1.rename(columns={'LSOA11CD':'LSOA code'})
london_proj_2 = london_proj_2.rename(columns={'LSOA11NM':'LSOA name'})
london_proj_2

,LSOA code,LSOA name,geometry
0,E01000001,City of London 001A,"POLYGON ((-0.09729 51.52158, -0.09652 51.52027..."
1,E01000002,City of London 001B,"POLYGON ((-0.08813 51.51941, -0.08929 51.51752..."
2,E01000003,City of London 001C,"POLYGON ((-0.09679 51.52325, -0.09647 51.52282..."
3,E01000005,City of London 001E,"POLYGON ((-0.07323 51.51000, -0.07553 51.50974..."
4,E01000006,Barking and Dagenham 016A,"POLYGON ((0.09115 51.53909, 0.09326 51.53787, ..."
...,...,...,...
4830,E01033742,Greenwich 007F,"POLYGON ((0.08240 51.49886, 0.08415 51.49803, ..."
4831,E01033743,Greenwich 002H,"POLYGON ((0.11081 51.50980, 0.11227 51.50782, ..."
4832,E01033744,Greenwich 007G,"POLYGON ((0.08072 51.49550, 0.08165 51.49491, ..."
4833,E01033745,Greenwich 002I,"POLYGON ((0.10806 51.50135, 0.10664 51.50021, ..."


In [16]:
#dwelling data
dwelling_1 = dwelling.loc[:,['Lower Layer Super Output Areas Code','Lower Layer Super Output Areas','Accommodation type (8 categories)','Observation']]
dwelling_1.columns = ['LSOA code','LSOA name','Accommodation type','Observation']
dwelling_1#dwelling data
# dwelling_1.to_csv('dwelling_1.csv',index=False,header=True)
dwelling_columns = ['Detached','Semi-detached','Terraced','purpose-built flats','converted or shared house','other converted building','commercial building','temporary structure']
dwelling_2 = pd.DataFrame()
dwelling_2['LSOA code'] = dwelling_1['LSOA code'].unique()
dwelling_2['LSOA name'] = dwelling_1['LSOA name'].unique()
for i in range(len(dwelling_columns)):
    dwelling_2[dwelling_columns[i]] = 0
p = 0
for q in range(dwelling_1.shape[0]):
    if q == 0:
        dwelling_2.iloc[p,((q%8)+2)] = dwelling_1['Observation'][q]
    elif q % 8 == 0:
        p = p + 1
        dwelling_2.iloc[p,((q%8)+2)] = dwelling_1['Observation'][q]
    else:
        p = p
        dwelling_2.iloc[p,((q%8)+2)] = dwelling_1['Observation'][q]
dwelling_2 #data set about number of each kind of dwelling type in each LSOA level region
##可能只用于筛选LSOA code

,LSOA code,LSOA name,Detached,Semi-detached,Terraced,purpose-built flats,converted or shared house,other converted building,commercial building,temporary structure
0,E01000001,City of London 001A,0,3,13,803,0,11,7,0
1,E01000002,City of London 001B,1,2,29,769,2,13,9,0
2,E01000003,City of London 001C,1,0,0,994,2,13,7,0
3,E01000005,City of London 001E,0,0,2,457,5,5,10,0
4,E01000006,Barking and Dagenham 016A,34,88,283,121,24,3,0,1
...,...,...,...,...,...,...,...,...,...,...
4989,E01035718,Westminster 019G,10,30,173,359,139,9,11,1
4990,E01035719,Westminster 021F,0,1,1,624,3,3,0,0
4991,E01035720,Westminster 021G,2,2,44,482,48,0,7,0
4992,E01035721,Westminster 023H,4,22,138,1084,29,7,22,1


In [35]:
code_o = ownership['geography code'].isin(dwelling_2['LSOA code'])
ownership_1 = ownership[code_o].sort_values(by=['geography code'])
ownership_2 = ownership_1.reset_index()
ownership_3 = ownership_2.drop(columns=['index','date'])
ownership_4 = ownership_3.rename(columns={'geography code':'LSOA code','geography':'LSOA name',
                                          'Tenure of household: Total: All households':'All households',
                                          'Tenure of household: Owned':'Owned',
                                          'Tenure of household: Owned: Owns outright':'Owns outright',
                                         'Tenure of household: Owned: Owns with a mortgage or loan':'Owns with a mortgage or loan',
                                          'Tenure of household: Shared ownership':'Shared ownership',
                                          'Tenure of household: Shared ownership: Shared ownership':'Shared ownership: Shared ownership',
                                          'Tenure of household: Social rented':'Social rented',
                                          'Tenure of household: Social rented: Rents from council or Local Authority':'Rents from council or Local Authority',
                                         'Tenure of household: Social rented: Other social rented':'Other social rented',
                                         'Tenure of household: Private rented':'Private rented',
                                          'Tenure of household: Private rented: Private landlord or letting agency':'Private landlord or letting agency',
                                         'Tenure of household: Private rented: Other private rented':'Other private rented',
                                          'Tenure of household: Lives rent free':'Lives rent free'})
#ownership_4既包含粗略的类别也包含更细化的类别的数据集
ownership_general = ownership_4.loc[:,['LSOA name','LSOA code','All households','Owned','Shared ownership','Social rented','Private rented','Lives rent free']]
ownership_general #按照较粗略的类别划分的ownership结果
ownership_detialled = ownership_4.loc[:,['LSOA name','LSOA code','All households','Owns outright','Shared ownership: Shared ownership','Shared ownership: Shared ownership','Other social rented','Private landlord or letting agency','Other private rented','Lives rent free']]
ownership_detialled #按照更细的类别划分的ownership结果
#ownership数据记录的是2021年的情况，只能假设2022年和2021年情况非常相近，算为limitation

,LSOA name,LSOA code,All households,Owns outright,Shared ownership: Shared ownership,Shared ownership: Shared ownership,Other social rented,Private landlord or letting agency,Other private rented,Lives rent free
0,City of London 001A,E01000001,837,337,4,4,4,280,49,1
1,City of London 001B,E01000002,826,284,2,2,4,326,38,0
2,City of London 001C,E01000003,1015,231,0,0,52,292,41,2
3,City of London 001E,E01000005,478,19,1,1,185,91,13,0
4,Barking and Dagenham 016A,E01000006,553,111,4,4,9,228,29,0
...,...,...,...,...,...,...,...,...,...,...
4989,Westminster 019G,E01035718,732,240,4,4,13,273,102,1
4990,Westminster 021F,E01035719,633,133,2,2,80,226,18,3
4991,Westminster 021G,E01035720,590,84,17,17,129,158,23,1
4992,Westminster 023H,E01035721,1307,168,6,6,425,356,44,6


In [36]:
##land use data
landuse_1 = landuse.drop(['MSOA code','MSOA name','House of Commons Library \nMSOA Names'],axis=1)
landuse_2 = landuse_1.iloc[13:33768,:]
code = landuse_2['LSOA code'].isin(dwelling_2['LSOA code'])
landuse_3 = landuse_2[code].sort_values(by=['LSOA code']) #all land use data in London based on LSOA level
landuse_3 = landuse_3.loc[:,['LSOA code','LSOA name','Community buildings','Leisure and recreational buildings','Industry','Offices','Retail','Storage and warehousing','Landfill and waste disposal','Minerals and mining','Institutional and communal accommo-dations','Residential','Highways and road transport','Transport (other)','Utilities','Agricultural buildings','Agricultural land','Forestry and woodland','Natural land','Rough grassland','Water','Total.10','Total.11','Unidentified building','Unidentified general manmade surface','Unidentified structure','Unknown','Total.12']]
landuse_4 = landuse_3.rename(columns={'Total.10':'Outdoor recreation'})
landuse_4 = landuse_4.rename(columns={'Total.11':'Residential gardens'})
landuse_4 = landuse_4.rename(columns={'Total.12':'Undeveloped land'})
landuse_5 = landuse_4.replace('-',0) #将原数据中“-”全部替换成0，假设“-”意味不存在这种土地利用类型
landuse_6 = landuse_5.reset_index().iloc[:,1:]
for i in range(len(landuse_6['Transport (other)'])): #单独转换这一列
    if type(landuse_6['Transport (other)'][i]) == type('1'):
        landuse_6['Transport (other)'][i] = int(landuse_6['Transport (other)'][i].replace(',',''))
landuse_6.iloc[:,2:] = landuse_6.iloc[:,2:].astype(int) #转换数据类型
landuse_6['sum_area'] = 0
for p in range(landuse_6.shape[0]):
    landuse_6['sum_area'][p] = landuse_6.iloc[p,2:].sum(axis = 0)
landuse_6#26种土地利用类型在每个LSOA级别地区所占面积，包括未知和未开发类型，单位是hectares
landuse_7 = landuse_6.copy()
for m in range(landuse_7.shape[0]):
    for n in range(2,landuse_7.shape[1]-1):
        landuse_7.iloc[m,n] = landuse_6.iloc[m,n]/landuse_6['sum_area'][m]
landuse_7 #data set contains properties of each kind of land use type in each LSOA level region
landuse_8 = landuse_7.copy()
landuse_8['entropy'] = 0.0
for a in range(landuse_7.shape[0]):
    for b in range(2,landuse_7.shape[1]-2):
        if landuse_7.iloc[a,b] == 0:
            landuse_8.iloc[a,b] == 0
        else:
            landuse_8.iloc[a,b] = landuse_7.iloc[a,b]*(math.log(landuse_7.iloc[a,b],math.e))
for c in range(landuse_8.shape[0]):
    landuse_8['entropy'][c] = -(landuse_8.iloc[c,2:29].sum(axis = 0)) #landuse_8 is the data set contains all types of land use in each LSOAlevel region with corresponding entropy
landuse_8

/tmp/ipykernel_116/3231418878.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  landuse_6['Transport (other)'][i] = int(landuse_6['Transport (other)'][i].replace(',',''))
/tmp/ipykernel_116/3231418878.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  landuse_6['sum_area'][p] = landuse_6.iloc[p,2:].sum(axis = 0)
/tmp/ipykernel_116/3231418878.py:23: RuntimeWarning: invalid value encountered in long_scalars
  landuse_7.iloc[m,n] = landuse_6.iloc[m,n]/landuse_6['sum_area'][m]
/tmp/ipykernel_116/3231418878.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

,LSOA code,LSOA name,Community buildings,Leisure and recreational buildings,Industry,Offices,Retail,Storage and warehousing,Landfill and waste disposal,Minerals and mining,...,Water,Outdoor recreation,Residential gardens,Unidentified building,Unidentified general manmade surface,Unidentified structure,Unknown,Undeveloped land,sum_area,entropy
0,E01000001,City of London 001A,-0.197304,0.000000,0.0,-0.197304,0.000000,0.0,0.000000,0.0,...,0.000000,-0.197304,0.000000,-0.287970,-0.287970,0.0,0.0,0.076923,13,-11.282718
1,E01000002,City of London 001B,-0.136326,-0.136326,0.0,-0.136326,-0.136326,0.0,0.000000,0.0,...,-0.136326,-0.136326,0.000000,-0.331751,-0.265680,0.0,0.0,0.043478,23,-20.947636
2,E01000003,City of London 001C,0.000000,0.000000,0.0,0.000000,0.000000,0.0,-0.298627,0.0,...,0.000000,-0.298627,0.000000,0.000000,-0.298627,0.0,0.0,0.166667,6,-4.673534
3,E01000005,City of London 001E,0.000000,0.000000,0.0,-0.160576,-0.244136,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,-0.298627,-0.244136,0.0,0.0,0.055556,18,-16.260148
4,E01000006,Barking and Dagenham 016A,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,-0.366204,0.000000,-0.180537,0.0,0.0,0.066667,15,-13.607497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,E01035718,Westminster 019G,-0.105103,0.000000,0.0,-0.020926,0.000000,0.0,0.000000,0.0,...,0.000000,-0.174424,-0.036660,-0.020926,-0.062936,0.0,0.0,0.007491,267,-266.219779
4990,E01035719,Westminster 021F,-0.259930,0.000000,0.0,-0.259930,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,-0.259930,-0.259930,0.0,0.0,0.000000,8,-6.332538
4991,E01035720,Westminster 021G,-0.321888,-0.230259,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,-0.230259,0.000000,-0.230259,0.0,0.0,0.000000,10,-8.304257
4992,E01035721,Westminster 023H,-0.149787,0.000000,0.0,0.000000,-0.149787,0.0,0.000000,0.0,...,0.000000,0.000000,-0.284568,-0.149787,-0.230259,0.0,0.0,0.050000,20,-18.252948


In [37]:
#spatial join airbnb data, combine airbnb data with London LSOA data 
airbnb_join = gpd.sjoin(airbnb_points, london_proj_2, op="within")
airbnb_join.loc[:,['id','LSOA code','LSOA name']]
airbnb_join_1 = airbnb_join.loc[:,['id','LSOA code','LSOA name']]
airbnb_LSOA = airbnb_join_1.groupby(['LSOA code','LSOA name']).count() #count number of airbnb listings in each LSOA level region
airbnb_LSOA.columns = ['listing_counts']
airbnb_LSOA_1 = airbnb_LSOA.reset_index()
airbnb_areas = london_proj_2.merge(airbnb_LSOA_1, on='LSOA code', how='outer')
airbnb_areas['area'] = airbnb_areas.area*10**6 #area of each LSOA level region
airbnb_areas['density'] = airbnb_areas['listing_counts'] / airbnb_areas['area']#count density of airbnb listings in each LSOA level region
airbnb_areas
airbnb_areas.to_csv('airbnb_areas.csv',index=False,header=True)

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_116/2673480416.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  airbnb_join = gpd.sjoin(airbnb_points, london_proj_2, op="within")
/tmp/ipykernel_116/2673480416.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  airbnb_areas['area'] = airbnb_areas.area*10**6 #area of each LSOA level region


In [54]:
#merge all data set together
merge_data = airbnb_areas.merge(landuse_8, on = 'LSOA code', how='left')
merge_data_1 = merge_data.loc[:,['LSOA code','LSOA name','listing_counts','density']]
merge_data_2 = merge_data.iloc[:,8:34].replace(',','')
merge_data_2 = merge_data_2.astype(float)
merge_data_3 = pd.concat([merge_data_1,merge_data_2],axis = 1)
merge_data_4 = merge_data_3.merge(ownership_4, on = 'LSOA code', how='left')
merge_data_4#最终数据集，但包含NA
merge_data_4.to_csv('merge_data_na.csv',index=False,header=True)

In [53]:
merge_data_5 = merge_data_4.drop(columns = 'LSOA name_y')
merge_data_5.iloc[:,2:] = merge_data_5.iloc[:,2:].replace([np.nan],0) #将所有na值都改为0
merge_data_6 = merge_data_5.rename(columns = {'LSOA name_x':'LSOA name'})#最终数据集，但不包含NA
merge_data_6.to_csv('merge_data.csv',index=False,header=True)